In [1]:
import numpy as np
import cv2 as cv
import glob
import matplotlib.pyplot as plt

In [2]:
x_corners, y_corners = (6, 8)

In [3]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

objp = np.zeros((x_corners*y_corners,3), np.float32)
objp[:,:2] = np.mgrid[0:x_corners,0:y_corners].T.reshape(-1,2)

objpoints = [] 
imgpoints = [] 

images = sorted(glob.glob('./zhang-data/*.png'))
num_images = len(images)
num = 1

for fname in images:
    print(f"Image {num}")
    print(fname)
    img = cv.cvtColor(cv.imread(fname), cv.COLOR_BGR2GRAY)

    gray = cv.resize(img, (1024, 1024))
    
    ret, corners = cv.findChessboardCorners(gray, (x_corners, y_corners), None)
    print(ret)

    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

        corners2 = corners2.reshape(-1, 2)
        np.savetxt(f'./zhang-coords/{num}.txt', corners2)
    num += 1

Image 1
./zhang-data/1.png
True
Image 2
./zhang-data/10.png
True
Image 3
./zhang-data/11.png
True
Image 4
./zhang-data/12.png
True
Image 5
./zhang-data/13.png
True
Image 6
./zhang-data/14.png
False
Image 7
./zhang-data/15.png
True
Image 8
./zhang-data/16.png
True
Image 9
./zhang-data/17.png
True
Image 10
./zhang-data/2.png
True
Image 11
./zhang-data/3.png
True
Image 12
./zhang-data/4.png
True
Image 13
./zhang-data/5.png
True
Image 14
./zhang-data/6.png
True
Image 15
./zhang-data/7.png
True
Image 16
./zhang-data/8.png
True
Image 17
./zhang-data/9.png
False


In [4]:
print(objpoints[0][:, :2])

[[0. 0.]
 [1. 0.]
 [2. 0.]
 [3. 0.]
 [4. 0.]
 [5. 0.]
 [0. 1.]
 [1. 1.]
 [2. 1.]
 [3. 1.]
 [4. 1.]
 [5. 1.]
 [0. 2.]
 [1. 2.]
 [2. 2.]
 [3. 2.]
 [4. 2.]
 [5. 2.]
 [0. 3.]
 [1. 3.]
 [2. 3.]
 [3. 3.]
 [4. 3.]
 [5. 3.]
 [0. 4.]
 [1. 4.]
 [2. 4.]
 [3. 4.]
 [4. 4.]
 [5. 4.]
 [0. 5.]
 [1. 5.]
 [2. 5.]
 [3. 5.]
 [4. 5.]
 [5. 5.]
 [0. 6.]
 [1. 6.]
 [2. 6.]
 [3. 6.]
 [4. 6.]
 [5. 6.]
 [0. 7.]
 [1. 7.]
 [2. 7.]
 [3. 7.]
 [4. 7.]
 [5. 7.]]


In [5]:
np.savetxt("./zhang-coords/world_points.txt", objpoints[0][:, :2])

In [6]:
imgpoints = np.array(imgpoints)
print(imgpoints.shape)

(15, 48, 1, 2)


In [7]:
print(imgpoints)

[[[[462.14203 544.1426 ]]

  [[507.67664 536.31305]]

  [[550.42993 528.79193]]

  ...

  [[610.99255 760.79974]]

  [[646.293   748.42755]]

  [[679.8452  736.8233 ]]]


 [[[343.69623 164.22551]]

  [[393.4648  166.07977]]

  [[440.63654 167.74782]]

  ...

  [[494.70032 422.67035]]

  [[533.0929  419.49994]]

  [[570.4636  416.22864]]]


 [[[417.73038 368.50302]]

  [[455.41864 367.85202]]

  [[489.82892 367.17105]]

  ...

  [[516.7227  609.2598 ]]

  [[543.5016  599.9582 ]]

  [[568.33514 591.5608 ]]]


 ...


 [[[258.8898  193.6202 ]]

  [[284.71274 193.879  ]]

  [[310.41748 194.06009]]

  ...

  [[342.1459  334.5885 ]]

  [[366.80612 335.02356]]

  [[391.54913 335.4697 ]]]


 [[[495.16714 464.5371 ]]

  [[517.2095  464.9073 ]]

  [[539.4159  465.44028]]

  ...

  [[560.33453 584.3441 ]]

  [[582.90466 585.2606 ]]

  [[605.4087  586.28534]]]


 [[[450.1609  534.99536]]

  [[496.8839  537.6427 ]]

  [[544.6978  540.7187 ]]

  ...

  [[592.30054 786.8951 ]]

  [[639.97614 793.382  

In [8]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [9]:
print(mtx)

[[1.00412159e+03 0.00000000e+00 6.80111818e+02]
 [0.00000000e+00 7.25415876e+02 4.95988016e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [10]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )

total error: 0.19597403375116165
